In [1]:
import pandas as pd
import numpy as np

# Data Cleaning & Manipulation
With Pandas, we can do more than just select data that is already there. 
We can add new columns to our datasets, apply functions, iterate thorugh each row in the dataframe, and more.

This is where we move from "pandas for exploring our data" to "pandas for getting our data ready to feed into models".

In [50]:
wine = pd.read_csv('data/wine_reviews/winemag-data_first150k.csv', index_col=0)
energy = pd.read_csv('data/energy/PJM_Load_hourly.csv', parse_dates=True, index_col=0)

### Dealing with Null Values:

In [4]:
print("Does the wine df contain nulls?", wine.isnull().any().any())

Does the wine df contain nulls? True


In [ ]:
print("Does the wine df contain nulls?", energy.isnull().any().any())

**Question:** why did we use `.any()` twice when asking this question?

What happens if we just use `.any()` once?

In [7]:
wine.isnull().any() # check NA in cols

True

In [ ]:
wine.isnull().any().any() # checks NA in values of cols

Now, we need to deal with the null values in the wine table.
How we deal with them will depend on the data type of the column in which null values are found, and how we hope to use the data.

If a wine doesn't have a region listed, we can still get a lot of information about it from things like its country. But, if a wine doesn't have a price listed, it's difficult to get much information out of it. 

So, if a wine has a null value for either of its regions, we will set it to be "unknown". But, if it has a null price, we will drop the row.

In [10]:
print("Shape of wine table before is: ", wine.shape)

null_country = wine.loc[wine.loc[:, "country"].isnull()]
print("Number of Null Countries is: ", null_country.shape)

Shape of wine table before is:  (150930, 10)
Number of Null Countries is:  (5, 10)


The 5 rows where a NaN value existed for country have been dropped.

In [ ]:
wine = wine.loc[~wine.index.isin(null_country.index)]
wine.shape

Now we'll do the same for rows with a null price.

In [14]:
null_price = wine.loc[wine.loc[:, "price"].isnull()]
null_price.shape

(13695, 10)

In [16]:
wine = wine.loc[~wine.index.isin(null_price.index)]
wine.shape[0]

137230

Wow! We lost a lot of rows getting rid of the null price rows.
Can you think of another way we could deal with null prices that doesn't mean losing almost 1/10 of our data? What are the pros and cons to using these other methods?

Now, for the other columns in which there are nulls, we can use `df.fillna()` to populate the null values with a value of our choosing. 

This is where we have to make some decisions. Improperly setting variables can have unintended consequences on any analysis we plan to do later using this data.

In [ ]:
wine.isnull().any()

The only columns where we still have nulls are regions and designations.
To avoid having to drop out more rows, we will use "fillna()".

In [18]:
wine.fillna("unknown", inplace=True)

We now have no null values in our dataset!

In [ ]:
wine.isnull().any()

In [ ]:
# visualizing the df to see rows where "unknown" has been inserted
wine.head()

### Creating new columns from existing ones
In pandas, it's easy to make a new column from existing ones. 

In [ ]:
wine.loc[:, "Region"] = wine.loc[:, 'region_1'] + "_" \
    + wine.loc[:, 'region_2']
wine.head()

#### Using a function + apply to make a new column:
Let's say we want to make a new column with an ID we can use to identify the records.
We can use a helper function + the apply method in pandas to apply it to all rows.
Apply looks through each row or column of a dataframe (depending on Axis set).

In [22]:
def add_unique_id(row):
    row["New_ID"] = (row['country'] + "_" + 
                     row['winery'] + "_" +
                     str(np.random.randint(1, 1000)))
    return row

In [23]:
wine = wine.apply(add_unique_id, axis=1) # axis=1 for all rows


In [24]:
wine.head()

,country,description,designation,points,price,province,region_1,region_2,variety,winery,Region,New_ID
0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz,Napa Valley_Napa,US_Heitz_890
1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,unknown,Tinta de Toro,Bodega Carmen Rodríguez,Toro_unknown,Spain_Bodega Carmen Rodríguez_773
2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley,Knights Valley_Sonoma,US_Macauley_104
3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi,Willamette Valley_Willamette Valley,US_Ponzi_291
4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,unknown,Provence red blend,Domaine de la Bégude,Bandol_unknown,France_Domaine de la Bégude_893


We can use the same approach to make a column for "great and cheap" wines, defined here as $50 or less and with a score >95.

In [25]:
def great_and_cheap(row):
    if (row['price']<50) & (row['points'] >= 95):
        x = 1
    else:
        x = 0
    return x

In [27]:
wine.loc[:, 'price'] = wine.loc[:, 'price'].astype("int")

In [28]:
wine.loc[:, "Great and Cheap"] = wine.apply(great_and_cheap, axis=1)

We can then use that boolean value to sort on the wines.

In [30]:
wine.loc[wine.loc[:, "Great and Cheap"]==1].head()

,country,description,designation,points,price,province,region_1,region_2,variety,winery,Region,New_ID,Great and Cheap
11,US,"From 18-year-old vines, this supple well-balan...",Estate Vineyard Wadensvil Block,95,48,Oregon,Ribbon Ridge,Willamette Valley,Pinot Noir,Patricia Green Cellars,Ribbon Ridge_Willamette Valley,US_Patricia Green Cellars_480,1
12,US,A standout even in this terrific lineup of 201...,Weber Vineyard,95,48,Oregon,Dundee Hills,Willamette Valley,Pinot Noir,Patricia Green Cellars,Dundee Hills_Willamette Valley,US_Patricia Green Cellars_759,1
20,US,Heitz has made this stellar rosé from the rare...,Grignolino,95,24,California,Napa Valley,Napa,Rosé,Heitz,Napa Valley_Napa,US_Heitz_444,1
24,US,"San Jose-based producer Adam Comartin heads 1,...",R-Bar-R Ranch,95,45,California,Santa Cruz Mountains,Central Coast,Pinot Noir,Comartin,Santa Cruz Mountains_Central Coast,US_Comartin_969,1
2439,US,This is possibly the most savory rendition of ...,Estate,96,40,California,San Luis Obispo County,Central Coast,Syrah,Stolo,San Luis Obispo County_Central Coast,US_Stolo_872,1


**Exercise:** Make a new column in the energy dataset, showing the energy in time T-1 for each time T. Hint: there is a built-in method in Pandas that can help with this, no complicated functions needed :)

In [41]:
energy.head()
energy.assign(time =energy.index.time)


,PJM_Load_MW,time
Datetime,,
1998-12-31 01:00:00,29309.0,01:00:00
1998-12-31 02:00:00,28236.0,02:00:00
1998-12-31 03:00:00,27692.0,03:00:00
1998-12-31 04:00:00,27596.0,04:00:00
1998-12-31 05:00:00,27888.0,05:00:00
...,...,...
2001-01-01 20:00:00,35209.0,20:00:00
2001-01-01 21:00:00,34791.0,21:00:00
2001-01-01 22:00:00,33669.0,22:00:00


In [ ]:
energy.assign(energy_time=energy['time']-1)

**Exercise:** Re-read the CSV for the wine data, and write a function to take care of all the nan values. Bonus points if it includes a test.

In [51]:
wine.isnull().any()

country         True
description    False
designation     True
points         False
price           True
province        True
region_1        True
region_2        True
variety        False
winery         False
dtype: bool

In [53]:
def clean_null(df):
    null_indexes = df.loc[wine.loc[:, :].isnull()]
    clean_df = df.loc[~df.index.isin(null_indexes.index)]
    return clean_df

In [61]:
# easy way
wine.dropna(inplace=True)
wine.isnull().any()

country        False
description    False
designation    False
points         False
price          False
province       False
region_1       False
region_2       False
variety        False
winery         False
dtype: bool

## Updates: Pandas 1.0
Pandas had already been a well-tested package used in development environments for quite some time. The official move to version 1.0 mainly just included some format fixes and 
- Dedicated types for String, boolean and NaN (before, just called "object")
- New documentation
